In [1]:
# Required library for project 
#!pip install wordcloud
#!pip install spacy
!pip install gensim


In [2]:
# Required packages 

import re as re  # For preprocessing
import nltk
#import spacy  # For preprocessing
import numpy as np 
import pandas as pd # For data handling
from time import time  # To time our operations
from nltk.corpus import stopwords
from collections import defaultdict  # For word frequency
from nltk.tokenize import word_tokenize
from gensim.models.phrases import Phrases, Phraser
from sklearn.feature_extraction.text import CountVectorizer

/Users/sukanagaraj/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [3]:
# Loading dataset using Pandas DataFrame.
df = pd.read_csv('indeed_job_dataset.csv')

type (df) 

pandas.core.frame.DataFrame

In [4]:
# Check to data shape, Total number of columns and row.
df.shape

(5715, 43)

In [5]:
# Get the summary of DataFrame.
list(df.columns)

['Unnamed: 0',
 'Job_Title',
 'Link',
 'Queried_Salary',
 'Job_Type',
 'Skill',
 'No_of_Skills',
 'Company',
 'No_of_Reviews',
 'No_of_Stars',
 'Date_Since_Posted',
 'Description',
 'Location',
 'Company_Revenue',
 'Company_Employees',
 'Company_Industry',
 'python',
 'sql',
 'machine learning',
 'r',
 'hadoop',
 'tableau',
 'sas',
 'spark',
 'java',
 'Others',
 'CA',
 'NY',
 'VA',
 'TX',
 'MA',
 'IL',
 'WA',
 'MD',
 'DC',
 'NC',
 'Other_states',
 'Consulting and Business Services',
 'Internet and Software',
 'Banks and Financial Services',
 'Health Care',
 'Insurance',
 'Other_industries']

In [6]:
# Job description
df['Description'].head(5)

0    [<p><b>POSITION SUMMARY</b></p>, <p>\r\r\nThe ...
1    [<p><b>What do we need?</b></p>, <ul><li>\r\r\...
2    [<ul><li>Validate, analyze, and conduct statis...
3    [<p>Full time</p>, <p>Washington, DC metro are...
4    [<ul><li>Assist in consultations with business...
Name: Description, dtype: object

In [7]:
pd.set_option('display.max_columns', 43)
df.head(5)

,Unnamed: 0,Job_Title,Link,Queried_Salary,Job_Type,Skill,No_of_Skills,Company,No_of_Reviews,No_of_Stars,Date_Since_Posted,Description,Location,Company_Revenue,Company_Employees,Company_Industry,python,sql,machine learning,r,hadoop,tableau,sas,spark,java,Others,CA,NY,VA,TX,MA,IL,WA,MD,DC,NC,Other_states,Consulting and Business Services,Internet and Software,Banks and Financial Services,Health Care,Insurance,Other_industries
0,0,Data Scientist,https://www.indeed.com/rc/clk?jk=6a105f495c36a...,<80000,data_scientist,"['SAP', 'SQL']",2,Express Scripts,3301.0,3.3,1.0,"[<p><b>POSITION SUMMARY</b></p>, <p>\r\r\nThe ...",MO,More than $10B (USD),"10,000+",Health Care,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0
1,1,Data Scientist,https://www.indeed.com/rc/clk?jk=86afd561ea8c6...,<80000,data_scientist,"['Machine Learning', 'R', 'SAS', 'SQL', 'Python']",5,Money Mart Financial Services,NaN,NaN,15.0,"[<p><b>What do we need?</b></p>, <ul><li>\r\r\...",TX,NaN,NaN,NaN,1,1,1,1,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2,Data Scientist,https://www.indeed.com/rc/clk?jk=e0aad317e6d45...,<80000,data_scientist,"['Data Mining', 'Data Management', 'R', 'SAS',...",9,comScore,62.0,3.5,1.0,"[<ul><li>Validate, analyze, and conduct statis...",OR,NaN,NaN,NaN,1,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
3,3,Graduate Studies Program - Data Scientist,https://www.indeed.com/rc/clk?jk=1cfdd9e391a63...,<80000,data_scientist,['Certified Internal Auditor'],1,Central Intelligence Agency,158.0,4.3,30.0,"[<p>Full time</p>, <p>Washington, DC metro are...",DC,NaN,NaN,Government,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1
4,4,Data Scientist I,https://www.indeed.com/rc/clk?jk=fec647775a21e...,<80000,data_scientist,"['Statistical Software', 'Time Management', 'R...",7,Federal Reserve Bank of Dallas,495.0,4.1,30.0,[<ul><li>Assist in consultations with business...,TX,NaN,"Less than 10,000",Banks and Financial Services,0,0,0,1,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0


In [8]:
#Get the dateset info

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5715 entries, 0 to 5714
Data columns (total 43 columns):
 #   Column                            Non-Null Count  Dtype  
---  ------                            --------------  -----  
 0   Unnamed: 0                        5715 non-null   int64  
 1   Job_Title                         5715 non-null   object 
 2   Link                              5715 non-null   object 
 3   Queried_Salary                    5715 non-null   object 
 4   Job_Type                          5715 non-null   object 
 5   Skill                             5483 non-null   object 
 6   No_of_Skills                      5715 non-null   int64  
 7   Company                           5611 non-null   object 
 8   No_of_Reviews                     4753 non-null   float64
 9   No_of_Stars                       4753 non-null   float64
 10  Date_Since_Posted                 5611 non-null   float64
 11  Description                       5413 non-null   object 
 12  Locati

## Data Cleaning

In [9]:
#Removing Duplicates

# As of now we are not removing skill. will check if needed will add here. TBD

df = df.drop_duplicates(subset=['Job_Title','Job_Type','Description','Location'])
print(df.shape)

(5355, 43)


In [10]:
df.isnull().sum()

Unnamed: 0                             0
Job_Title                              0
Link                                   0
Queried_Salary                         0
Job_Type                               0
Skill                                217
No_of_Skills                           0
Company                               73
No_of_Reviews                        888
No_of_Stars                          888
Date_Since_Posted                     73
Description                          225
Location                             216
Company_Revenue                     3481
Company_Employees                   2359
Company_Industry                    1778
python                                 0
sql                                    0
machine learning                       0
r                                      0
hadoop                                 0
tableau                                0
sas                                    0
spark                                  0
java            

In [32]:
#Before we detect the skills from description , we need to standardize the description features

#Converting the Descrition feature into String type
df['Description']=df['Description'].apply(str)

#Removing tags from description
def clean_up_job_description(raw_data) :
    
    #Convert to lower case
    raw_data = raw_data.lower()
    
    #Remove Special Characters
    result = re.sub('(\\d|\\W)+','',raw_data)
    
    #Remove Tags
    result = result = re.sub('<.*?>','',raw_data)
    
    # Remove brackets.
    result = result.replace('[', '')
    result = result.replace(']', '')
    result = result.replace(')', '')
    result = result.replace('(', '')  
    
    result = result.replace('\r','')
    
    # Remove new line.
    result = result.replace('\n','')
    
    # Stripping first and last white space. 
    result = result.strip()
    
    return result


df['Description'] = df['Description'].apply(lambda raw_data : clean_up_job_description(raw_data))
df['Description'].head()

0    position summary, the business analyst role is...
1    what do we need?, you to have an amazing perso...
2    validate, analyze, and conduct statistical ana...
3    full time, washington, dc metro area, starting...
4    assist in consultations with business partners...
Name: Description, dtype: object

In [33]:
# Total number of null value.
df.isna().sum()
# Here we need to drop out the below columns 

Unnamed: 0                             0
Job_Title                              0
Link                                   0
Queried_Salary                         0
Job_Type                               0
Skill                                217
No_of_Skills                           0
Company                               73
No_of_Reviews                        888
No_of_Stars                          888
Date_Since_Posted                     73
Description                            0
Location                             216
Company_Revenue                     3481
Company_Employees                   2359
Company_Industry                    1778
python                                 0
sql                                    0
machine learning                       0
r                                      0
hadoop                                 0
tableau                                0
sas                                    0
spark                                  0
java            

In [34]:
#Skills and Description are important parameters in the model,hence there shouldnt be any null value for these feartures
# Will check in second pahse for re-fill data.
# df = pd.DataFrame(df)
# df.dropna(inplace = True)
# df.reset_index(inplace = True, drop = True)

In [35]:
df.isna().sum()

Unnamed: 0                             0
Job_Title                              0
Link                                   0
Queried_Salary                         0
Job_Type                               0
Skill                                217
No_of_Skills                           0
Company                               73
No_of_Reviews                        888
No_of_Stars                          888
Date_Since_Posted                     73
Description                            0
Location                             216
Company_Revenue                     3481
Company_Employees                   2359
Company_Industry                    1778
python                                 0
sql                                    0
machine learning                       0
r                                      0
hadoop                                 0
tableau                                0
sas                                    0
spark                                  0
java            

In [36]:
df.shape

(5355, 43)

In [37]:
# NLP pre-processing.

In [38]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 
stop = stopwords.words('english')

df['Description'].apply(lambda x: [item for item in x if item not in stop])

docs = df['Description']

#instantiate CountVectorizer() 
cv=CountVectorizer(max_df=0.95,max_features=10000,ngram_range=(1,3)) 
 
# this steps generates word counts for the words in your docs 
word_count_vector=cv.fit_transform(docs)
# word_count_vector.shape

[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1123)>
[nltk_data] Error loading wordnet: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1123)>
[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1123)>
[nltk_data] Error loading averaged_perceptron_tagger: <urlopen error
[nltk_data]     [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify
[nltk_data]     failed: unable to get local issuer certificate
[nltk_data]     (_ssl.c:1123)>


In [39]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer=TfidfTransformer(smooth_idf=True,use_idf=True) 
tfidf_transformer.fit(word_count_vector)

# print idf values 
df_idf = pd.DataFrame(tfidf_transformer.idf_, index=cv.get_feature_names(),columns=["idf_weights"]) 
 
# sort ascending 
df_idf.sort_values(by=['idf_weights'])

,idf_weights
and,1.051333
in,1.066382
of,1.067380
with,1.068580
to,1.071985
...,...
robinson,7.640063
hallmark,7.794213
fedex,7.794213
predictive science,7.794213


In [40]:
print(df['Description'].dtypes)
sent = [str(row).split() for row in df['Description']]
#print (sent)
phrases = Phrases(sent, min_count=30, progress_per=10000)

object


In [41]:
# Debugging 

sentences = sent #bigram[sent]
word_freq = defaultdict(int)
for s in sentences:
    for i in s:
        word_freq[i] += 1
len(word_freq)

88869

In [42]:
sorted(word_freq, key=word_freq.get, reverse=True)[:10]

['and', 'to', 'of', 'the', 'data', 'in', 'with', 'a', 'or', 'for']

In [43]:
word_freq

defaultdict(int,
            {'position': 1651,
             'summary,': 217,
             'the': 45461,
             'business': 11558,
             'analyst': 1055,
             'role': 1485,
             'is': 11122,
             'primary': 377,
             'architect': 148,
             'of': 46556,
             'reporting': 1890,
             'and': 123490,
             'dashboard': 122,
             'solutions': 3744,
             'for': 20666,
             'internal': 1928,
             'external': 1208,
             'clients.': 183,
             'utilizing': 433,
             'esi': 3,
             'corporate': 347,
             'standard': 424,
             'development': 4022,
             'tools': 3620,
             'this': 4447,
             'responsible': 1164,
             'design,': 1231,
             'development,': 991,
             'implementation,': 160,
             'analysis,': 1808,
             'interpretation': 131,
             'communication': 2421,
         

In [44]:
import multiprocessing
from gensim.models import Word2Vec
cores = multiprocessing.cpu_count()

In [45]:
#min_count = int - Ignores all words with total absolute frequency lower than this - (2, 100)
#window = int - The maximum distance between the current and predicted word within a sentence. E.g. window words on the left and window words on the left of our target - (2, 10)
#size = int - Dimensionality of the feature vectors. - (50, 300)
#sample = float - The threshold for configuring which higher-frequency words are randomly downsampled. Highly influencial. - (0, 1e-5)
#alpha = float - The initial learning rate - (0.01, 0.05)
#min_alpha = float - Learning rate will linearly drop to min_alpha as training progresses. To set it: alpha - (min_alpha * epochs) ~ 0.00
#negative = int - If > 0, negative sampling will be used, the int for negative specifies how many "noise words" should be drown. If set to 0, no negative sampling is used. - (5, 20)
#workers = int - Use these many worker threads to train the model (=faster training with multicore machines)

#gensim.models.Word2Vec(data, min_count = 1, size = 100,window = 5, sg = 1)

w2v_model = Word2Vec(min_count=20,
                     window=2,
                    # size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

In [46]:
t = time()
w2v_model.build_vocab(sentences, progress_per=10000)
print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

Time to build vocab: 0.02 mins


In [47]:
t = time()
w2v_model.train(sentences, total_examples=w2v_model.corpus_count, epochs=30, report_delay=1)
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))

Time to train the model: 0.68 mins


In [77]:
'''company_keyword_seed = ['google', 'locations','adobe','florida','san francisco','washington'
'Government'',Education and Schools',
'Energy and Utilities',
'Aerospace and Defense',
'Transport and Freight',
'Organization',
'Telecommunications',
'Retail',
'Computers and Electronics',
'Consumer Goods and Services',
'Restaurants, Travel and Leisure',
'Media, News and Publishing',
'Pharmaceuticals',
'Industrial Manufacturing',
'Auto',
'Real Estate',
'Human Resources and Staffing' ]
'''

company_keyword_seed =['Express Scripts','Money Mart Financial Services','comScore','Central Intelligence Agency',
                       'Federal Reserve Bank of Dallas National Security Agency','NYC Careers','OM Partners','University of Idaho','usajobs.gov','The University of Pittsburgh Rice University',
                       'Elev8 Hire Solutions','Catalina Marketing','Florida Polytechnic University','Achievement Network (ANet) Deloitte',
                       'ExxonMobil','MIT','Ochsner Health System','Raytheon','ERM Group','Google','Lawrence Livermore National Laboratory','Adobe Intuitive Surgical','University of Minnesota','Splunk','Southern Methodist University','Microsoft Duke University and Duke University Health System','Coyote Logistics','Twitter','iD Tech','Canvas Technology The Church of Jesus Christ of Latter-day Saints','Riverside Research','OpenAI','IBM','Asendia Management SAS','T-Mobile','Walmart UES','NextEra Energy, Inc.','Argonne National Laboratory','Elder Research Inc','Dell',
                       'Hallmark','Allstate L. L. Bean']
                       
                       
'''            
                    'University of California San Francisco','Wunderman','MGM Resorts International','Interactive Brokers','Grubhub Amica Insurance Company','UPMC','Hearts and Science','Operation Smile','Sedgwick Claims Management Services','MetLife'
                       ,'Code Pilot','Talking Rain Beverage Company','Bright Cellars','MetroStar Systems','Omatic Software Schnucks','Abbott Laboratories','Lawrence Berkeley National Laboratory','Shift Technology','Consumers Energy Penn State University','VillageCare','Intercontinental Exchange','Palo Alto Networks','The Rawlings Group','Devoted Health Caris Life Sciences','ADT Corporation','Thomas Jefferson University and Hospitals','MassMutual Financial Group FourthWall Media','BB&T','AbbVie','National Renewable Energy Laboratory','Sandhills Publishing',
                       'Energy Sense Finance New York Media LLC','ESAC Inc','Aunalytics','U.S. Bank','Vistra Energy','Precision for Medicine','Teradata','Greene, Tweed & Co.','PNC TXU Energy','Stryker','Nippon Dynawave Packaging','Smith Hanley Associates','BASF','ADT Security Services Logistics Management Institute','Johns Hopkins','Shippensburg University','CareSource','Universal Consulting Services','DISH Commerce Bank','Beyond Limits','Prime Therapeutics','UCS Consulting','Ball Corporation','Anthem, Inc.','AT&T','West Corporation Parkland Health & Hospital System','Mayo Clinic','SunTrust','Amgen','K12','Broad Institute','Intel','UCHealth','Foxconn','Octic Capital Department of Veterans Affairs','New Mexico State University','USAA','Accenture','WellCare','Banner Health American Axle & Manufacturing, Inc.','Cyient','Swagelok','MEI Technologies','XPO Logistics, Inc.','Citi Universities Space Research Association','Lubrizol','eBay Inc.','KPMG LLP','Childrens Medical Center'
                       ,'AvalonBay Communities Cummins Inc.','Cummins Power Systems, LLC.','JB HUNT','Ford Motor Company','Anne Arundel Medical Center','Comrise','Highmark Health Central California Alliance for Health','Booz Allen Hamilton','Lockheed Martin Corporation','Cognizant','Jobspring Partners VincentBenjamin','Travelers','HX5','TFS','CarMax','Talent Plus','Bertelsmann','Forcepoint','Diverse Lynx','RubrYc Therapeutics L3 Technologies','Methodist Le Bonheur Healthcare','Arcadis','Smiths Detection','Orbital Insight','GE Careers The Aerospace Corporation','Odyssey Systems Consulting Group','Simons Foundation','Harvard University','First Horizon Stratacache, A Family of Companies','Axius Technologies','SigOpt','Kaleido Biosciences','Chenega Corporation','Foreground Security Electronic Arts','Miami HEAT','Garmin','Cisco Systems','Day Zero Diagnostics','GM Financial','Nike','EagleView Technologies University of Massachusetts Medical School','Standard Insurance Company','The Hanover Insurance Group Hewlett Packard Enterprise','Prime Solutions Group (PSG)','Weatherford','Next Rev Technologies LLC','Triquetro','ClimaCell','Sprint Synchronoss Technologies, Inc.','Synchronoss','TVG Network Betfair US','Northrop Grumman','Domino's','Cook Medical','Simple CrossFit, Inc.','Comcast','Zurich North America','Gayathri's Sandbox','Pros.','Rifiniti','Arable Labs, Inc.','Zumper','ISE Data Systems Grid Dynamics','Freds','Zillow Group','Nordstrom','HNI Allsteel','Lexmark International, Inc.','Florida Blue','Equifax','Apeel Sciences RELX Group','Andersen Corporation','WestRock','ShipBob Inc','Nielsen','Bosch Group','Xylem','Crossover','NT Concepts','Goodyear Lake Trust Credit Union','Ecolab','Spreetail','TracFone Wireless','eQHealth Solutions','HOMER','Shell','Barri Financial Group','Plaid Vantiv','Trinity Health','AnswerRocket','Northwell Health','Novartis','WeddingWire','US Pharmacopeia','Corning','CGG','Nexidia','Netflix HNI Corporation','Wayfair','PPL Corporation','Seagate Technology','Preferred Resources Inc.','Tek Experts','TriHealth Kaiser Permanente','EMD','P3 North America','Colgate-Palmolive','Staples','The Home Depot','Kantar Careers','Genmab','Walmart eCommerce Formation','Aetna','Aptiv','Schneider National','Press Ganey Associates Inc.','Novetta','Capgemini','HotelTonight','Capax Global LLC', 'AmTrust Financial Services','LendingTree','Ulta Beauty','W.R. Berkley','FinLocker Pearson','Zimmerman Advertising','Leidos','PACCAR','SYSCO','McKinsey & Company','Zachary Piper Solutions','Novelis','devwrx','FleetPride JPMorgan Chase','Klaviyo','Memorial Health System','Crossover Health','Feedzai','Virginia Tech Applied Research Corporation Matrix Medical Network','Niantic, Inc.','Niantic Labs','CenturyLink','Optoro','Bold','Quadrint, Inc.','ActiveCampaign Alaka`ina Foundation Family of Companies','Motorola Solutions','Yamaha','Modern Meadow','Qikspace','CACI','Snap Inc. Knowledge Facilitation Group','Toptal','Subaru of America','Wiley','DTCC','SAP','Verizon','Pluralsight','HCSC Pacific Northwest National Laboratory','IEEE GlobalSpec','AF Group','Accident Fund Holdings, Inc.','Ericsson','Radiant Solutions Micron','Viral Launch, Inc.','Fred Hutchinson Cancer Research Center',
                       
                       
                       'Martins Point Health Care','Carlisle & Company','Qwinix Amne','Colony Brands, Inc.','BAE Systems','Paypal','Carolinas HealthCare System','UPS','Ascena Retail Group, Inc.','Verra Mobility Illumination Works','C.H. Robinson Worldwide, Inc.','MEDSTAR HEALTH RESEARCH INSTITUTE (MHRI)','Albemarle Corporation Lincoln Financial Group','U.S. Venture, Inc.','WWE','Brillio','Karsun Solutions, LLC','Milliman','SanofiUS','Sovrn Holdings','UL LLC Applied Systems Inc.','Delaware North','Lovepop','Trend Micro','Harbor Wholesale Foods','Cardinal Solutions Group Ascensia Diabetes Care','Smartly','MarketDial','Mid-Atlantic Permanente Medical Group','Cardinal Health','Paychex Inc.','Cerebri AI Oath Inc','Numerator','Entergy Corporation','StubHub','Quicken Loans','Centene','Underwriters Laboratories','Prokarma Inc. Tractor Supply Company','Marsh & McLennan Companies','Radian Group Inc.','SumTotal Systems','Facebook','Ultimate Software Ideal Concepts, Inc.','World Fuel','Selective Insurance Group','Monsanto','Vettery','Ascension Healthcare','Dun & Bradstreet The J. M. Smucker Company','Procter & Gamble','City of Seattle','Two Sigma Investments, LLC.','Exact Sciences Corporation Driven Brands','Falls Lake Insurance Companies','Eli Lilly','Pop Healthcare LLC','Skillsoft','Tesla','iHeartMedia, Inc.','3Q Digital Ipsos North America','Exact Sciences','Vanda Pharmaceuticals Inc.','American Chemical Society','SHI International Corp. Medtronic','Precima','UTC','University of Maryland Medical System','DEG','Groupon','elicit','Conagra Brands','Progressive Leasing Thermo Fisher Scientific','Kaplan Test Prep','Kaleidoscope','Tilt Lending','Humana','Hawaiian Airlines','IT Concepts Inc. BAIN & COMPANY','BRS','LogMeIn','Integral Consulting Services, Inc.','DoubleDown Interactive LLC','Essence','SpartanNash First National Bank of Omaha','Visa','ZestFinance','Centura Health','Red Ventures','Barings','Kaplan','Cigna','PRA Health Sciences','ABSc HP','Huawei','BLUEHAWK, LLC','University of Utah','E*TRADE FINANCIAL','Ally Financial','Advantage SCI','Civis Analytics','Maana Equityzen','The Cadmus Group, Inc.','Society for Human Resource Management','Annalect','Northern Trust Corp.','Hitachi Vantara','EXL American Family Insurance','Huntington Bank','Cambia Health','Alion Science and Technology','NORC at the University of Chicago Morning Consult','First National of Nebraska, Inc.','Boll & Branch','Slack','Auto-Owners Insurance','SpaceCurve','Hertz','Welltower TripleLift','Alliant Credit Union','ServiceMaster','PwC','Guidehouse','Visionist, Inc.','Cormac Corporation','SAS Institute','Dexcom Sabre','TopSchoolJobs.org','Oracle','Ingersoll Rand','Axiologic Solutions','Celgene','Tsource','DataLab USA','Drybar','Philips Houston Methodist','Regions Bank','Bristol-Myers Squibb','Mars','BlueLabs Analytics','ConAgra Foods ManTech International Corporation','Engility','Analysis Group','RBC','Medical Science & Computing, Inc. Medical Science & Computing','KAR Auction Services, Inc.','UBS','Fidelity Investments','Agilent','Precocity','MetroHealth RetailMeNot, Inc.','Vivid Seats','Pfizer Inc.','CALNET INC.','Bio-Rad','Northwestern Mutual','Dotdash','Formativ Health','Pfizer Nabler Web Solutions','Varian Medical Systems','Blue Cross Blue Shield of Michigan','KeyBank','Assured Consulting Solutions H2M Group','Kemper Corporation','Enable Midstream','MedStar Health','Experian','Grant Thornton','Experis','Arrayo Brighthouse Financial','AncestryDNA','National Fish Wildlife Foundation','Molekule','Cubic Corporation MedStar National Rehabilitation Network','Tranzact','Aret√© Associates','MasterCard','Harnham','ASET Partners Camp4 Therapeutics Corporation','Lenovo','Jacobs','Redfin','Claritas','KEYW Corporation','Inabia Solutions and Consulting Wells Fargo','Lorven Technologies','OGSystems','Workbridge Associates','Altamira Technologies Corporation','Digital Promise','Bixal Flagship Pioneering','California State University','Intelligent Automation','Solidus Technical Solutions','DNAnexus','JD.com Obsidian Solutions Group','Butterfly Network','Mosaic North America','Early Warning Services','Panasonic','American Express Navstar, Inc.','Lightmatter','Wolters Kluwer','Open Systems Technologies, Inc.','Fifth Third Bank','Rockwell Collins System1 Biosciences','Battelle','eHire, LLC','1st Solution USA','Endurance International Group','Spotify','ViaSat','Foot Locker Ace Technologies','Salesforce','Merck','Norfolk Southern Corp','Eventbrite','Beyondsoft Consulting','Applause','Arthur Lawrence','Square Trimble Inc.','Genedata','Directly','Tradesy','InVision Studio','Time Warner','Siemens','HealthGrades','Edison Energy','Recorded Future SpotX','KBRWyle','KMM Technologies','Transamerica','Snowflake Computing','TripAdvisor','Pitney Bowes','HumanTouch, LLC','Vevo','b.well Legends','MachineZone','Numerdox','citius tech','HyperspaceVentures','Creative Alignments','Williams-Sonoma, Inc.','NetMotion Software Sartorius Corporation','NewYork-Presbyterian Hospital','Trimble','Fora Financial LLC','GlaxoSmithKline','ConnectYourCare Clarivate Analytics','Q2ebanking','Bigfoot Biomedical','State Farm','Brightidea','Catasys','Verb Surgical','MapR','Tessella','Uber PARC, a Xerox company','Wolverine Trading','Dana-Farber Cancer Institute','myCOI','XL Catlin','Country Financial','Centro Impetus Technologies','HouseCall Pro','Huobi','QxBranch','GlassDoor','BMW North America','NBA Properties','Parkview Health','Bose Taboola','Edelman','Watts Water Technologies','ThinkIQ','Upstart','Conversant Media','Calico','CIITS','Nokia','MacAulay-Brown, Inc. (MacB) Ticketmaster','Dropbox','PepsiCo','Mitchell International, Inc.','Etsy','Gartner, Inc.','Pray','Weber Shandwick','Disney','Phreesia Integral Ad Science','Veritone','Opendoor','Aera Technology','Owens Corning','Protective Life Corporation','Arconic','GoDaddy Coupa Software','FedEx','Bind Benefits','Voya Financial','Alere','Xandr','DigitasLBi','Stanley Black & Decker','Lab126','Fair','Houzz Asurion','ACI Worldwide','Predictive Science','NEURA','Unisys','Kogentix','MOBE, LLC','AllianceData','Epsilon','TrueCar, Inc. The TJX Companies, Inc.','Maxim Integrated','SoFi','Church Pension Group','Jewelers Mutual Insurance Company','Esurance Harley-Davidson','BlackLine','Slalom Consulting','Chatmeter','Roche','Wish','Par Government Systems Corporation','OpenTable','Honeywell Atos','Precision Health AI','Domo, Inc.','Coso IT','RS Energy Group','ZF','Realogy','3M','St. Jude Children's Research Hospital EPAM Systems','S&P Global','Rover','Jet.com','Syntelli Solutions, Inc','Improbable','Clarabridge','Honda Research Institute USA Applied Materials Inc.','Manulife','Public Company Accounting Oversight Board','McGraw Hill Financial','NetApp','Mitre Corporation Sage Intacct','Cyberspace Solutions','Blue Nile','Aspen Technology','Amches, Inc.','BoxyCharm','Lowe's','DuPont','Invisibly The Climate Corporation','Upwork','Hackensack Meridian Health','United States Steel','Onyx Government Services, LLC Change Healthcare','Pivotal Commware','CallMiner, Inc','MSA Safety','Proofpoint','OneKreate','Vituity','Hilton','Convey Marketers on Demand Inc.','Akili Interactive','L&T Technology Services Ltd.','Satsyil Corporation','Convey, Inc. HERE Technologies','Signifyd','Esri','Sabio Mobile','Candid Co.','First Tech Federal Credit Union','Faire','Sentieo Planned Systems International','Remedy BPCI Partners, LLC.','Hagerty','Convoy','Strava','Remedy Partners','Vertex Vertex Pharmaceuticals','Engie','Thrive Market','Morgan Stanley','Plymouth Rock Assurance Corporation','IQVIA','Eagle Ray Inc Volanno','Kinetica DB','Prudential','First American Corporation','Picarro','Immuta','Liberty Mutual','Qualtrics','Illumina BMO Financial Group','Kinsa Inc.','Sartorius North America','Intuit','ExtraHop Networks, Inc.','Cityblock Health HR, Legal, & Administration','Ellie Mae','WellSky','Johnson & Johnson Family of Companies','Quartet','Abiomed','Affirm','Alqimi Twentieth Century Fox','Maven Wave Partners','Conduent','Cognitive Scale','SHINE Systems & Technologies','Crowe','AXIS Insurance Criteo','Big Fish Games','Genentech','MSA Safety Inc.','Xactly Corporation','Coherent, Inc.','bluebird bio','FabFitFun Varen Technologies','Coherent Germany','FM Global','All-In Analytics','HHB Systems','BCG Digital Ventures Benson Hill Biosystems, Inc','MultiScale','zulily','High Alpha','Quora','ICR','Blackbaud','Castlight Health','KAYAK Bank of America Merchant Services','Elite SEM','EY','Heartland Dental, LLC','Verisk Analytics','Pragmatics','Vanguard','Bartech Group Berico Technologies','Peraton','Clara','Invesco','Zscaler','American Institutes for Research','Funding Circle US','First Orion Social Solutions','Apogee Integration, LLC','Productive Data Solutions, Inc.','Everlane','Regis Corporation','PetSmart Service Management Group, Inc.','General Dynamics Mission Systems','Clockwork Solutions','Dynetics','Amplitude New York Life Insurance Company','Altair Engineering','Ross-Carlisle Group','Eaton Vance','NVIDIA','Technology Partners BNP Paribas','FICO','Toyota Research Institute','Scaleapi','Liquidnet','2K Games','Ginger.io','Zynga','Symantec','Yelp','Globant Rockstar New York','Restoration Media','Bluecore Inc.','Nutanix','Kelvin','iHerb','Prospect 33','Spokeo','Academia.edu','John Deere Daugherty Business Solutions','YourMechanic','First Republic Bank','AppZen','Ubisoft','Optimizely','LeapYear','Reorg Research Altice USA','Autodesk','LimeBike','Quantcast','DeepCurrent Technologies, Inc.','C3','Neuberger Berman','Liberty Lending','Crowdstrike ServiceTitan','Porsche','VROOM','Sealed Air','Edison Software, Inc.','Rockstar San Diego','Nauto','Gap Inc.','Applied Memetics LLC Lark Health','Blue Owl','Ryder','Quantiply Corporation','Internet Brands','Mist Systems','Helpshift','Viome','ThirdLove','Crystal Dynamics Peak6','Grand Rounds','Yapstone','Aruba Networks','Credit Sesame','Valassis Digital','Uhana, Inc.','EMIDS','Xilinx','Veeva The Carian Group','AIG','Barrick Gold Corporation','Photon','Sony Interactive Entertainment PlayStation','Omnicell','Waymo','Coursera Scalable Digital','Suplari','Tonal','Pocket Gems','Coinbase','Emailage','Zazzle','Otsuka','PG&E','Thunder Token','Scoot Networks','PlaceIQ Franklin Templeton Investments','WW International (formerly Weight Watchers)','Porch','Lumos Labs, Inc.','Polis, Inc. MetroPlus Health Plan','Lieberman Research Worldwide','Safeway Corp.','Lyric','Wealthfront','Dollar Shave Club','Farfetch','Qualys Sirius XM Radio','Perficient','Acuity, Inc.','Atlassian','Infoblox','Research Innovations Inc','General Mills','Lattice Engines Nift Networks','Nerdwallet','Peloton','Goldman Sachs','Tesorio','JUUL Labs','Zot Inc.','BlackRock','Proteus Digital Health','Payette Group Guidewire Software, Inc.','Betterment LLC','GroupM','Maxus','Western Digital','Nanigans','Covance','OneGlobe','Galvanize','Coverent University of California San Francisco Medical Center','JM GROUP','Boeing','The Boston Consulting Group','Sitecore Fluid Intelligence','LabCorp','VMware','Capital Group Companies','Visual Awareness Technologies and Consulting, Inc. Powertek Corporation','Cell Signaling Technology, Inc','Ascent Services Group Redfish Technology - High Tech Executive Recruiters','Turnitin, LLC','Darwin Recruitment','Kaizen','Axelon Services Corporation Chime','Clarity Insights','IQ Workforce','BAE Systems Applied Intelligence','Solving It','stanleyreid','Quest Diagnostics Stride Search','CRGT Inc.','Avanade','NBCUniversal','Dice','3coast','Lionbridge Technologies','Signify','Blue Origin','Medallia, Inc. Stella.ai','TIBCO Software','Discovery Communications, LLC','7Park Data','Narvar','Keep Truckin','David Weekley Homes Moody's Corporation','Linc Global, Inc.','Figure','Alibaba','FoxNext Games','DRW Trading Group','DRW','ServiceNow','Motif Investing CircleUp','Samsung Semiconductor, Inc.','Zingbox','Workday','Synechron','Taino Consulting Group','Parallel HR','Intelligent Waves Llc HeartFlow','HD Vest','JLL','NetBrain Technologies, Inc.','Publicis Spine','Gradient.io','aThingz','ShareThis, Inc','Roku','Datadog DataRobot','Octane Lending','PTP','Juvo','Blume Global','securonix','FireEye','Plume','Cogitativo','QuantumBlack','Unity Technologies','ACS Socure','Averity','TWINN INTELLIGENCE GROUP','Prognos','The Hagan-Ricci Group, Inc.','Trace3','Daley and Associates','Health Catalyst AltaSource Group','CRB Workforce LLC','Invictus International Consulting, LLC','Integrated Management Resources, LLC','Pilot AI Analytic Recruiting','Advanced Micro Devices, Inc.','Great Bay Staffing Group','Lyft','Optimove','Commonwealth of Massachusetts UC DAVIS HEALTH SYSTEM','Columbia University','Bryant & Stratton College','Bloomberg','Servient Inc','HopJump Colliers International','Freedom Forever','Shoes For Crews, Inc.','AVID CENTER STAFF','Tampa Family Health Centers','FIS Wellmore, Inc.','Arkansas Blue Cross and Blue Shield','Youth Policy Institute','McGraw-Hill Education','XLA Lubbock Heart & Surgical Hospital','ZEISS Group','Zeiss','Lockton, Inc.','Boyd Caton Group (bcg)','GOCOOL BHP Engineering & Construction','Levy','New Visions Central Office','City of Minneapolis','Blade','Avkare Inc','The Carle Foundation Arkatechture','Raymond James','BBYO -','Stanford University','Cota','New Visions for Public Schools','Canton-Potsdam Hospital JetBlue Airways Corporation','NYSTEC','National Grid','Resolvit, LLC','Goodwill Industries International, Inc. (GII) Glassview Media','STOBER Drives, Inc.','General Dynamics Information Technology','Advance Auto Parts','23andMe','Healthtronics','DHL Progrexion Holdings Inc','Lytx','Southern California Edison','Fortive','doTERRA International','Abilene Christian University London Stock Exchange Group','Goby','Georgia Military College','CDW','Rational Consulting','Boss Fight Entertainment Butterball, LLC','Clarity USA','Mixbook','Memorial Sloan Kettering','King County Housing Authority','ZIN TECHNOLOGIES INC.','Masco The Wonderful Company LLC','eSpark','Health One Alliance, LLC','Teacher Retirement System of Texas','Emory University Major League Soccer','Brown & Brown Insurance','Fidelity National Financial','City of Tucson','SCAN Health Plan Alegeus Technologies','Ciox Health','CMI Media and Compas, Inc.','Graphic Products','Cresco Labs','AltaMed','Regional One Health Alltran','Logic Rule','Lutron Electronics','jump ramp','First National Bank of America','Outreach Process Partners','ECS Federal LLC Sentry Insurance','WebMD','Johns Hopkins University','AECOM','Office of the Nassau County Comptroller','IPRO','Zeta Global The Dow Chemical Company','GeoDigital','Sigma Space','Keck Medical Center of USC','Fruition Partners','IXIS Alliance Health Professionals','Resource Systems Group, Inc.','CompuCom','Moda Health','Germania Insurance','stearns bank Technicolor','Spy Pond Partners','Pinkerton','Ukpeagvik I√±upiat Corporation/Bowhead Family of Companies','NextGen Healthcare UNITED NEGRO COLLEGE FUND','Fitch Group','Patagonia, Inc','USfalcon','Texas Health and Human Services Commission','The Motley Fool Federal Management Systems, Inc.','Care New England Health System','U.S. Anesthesia Partners','Progressive Alameda Health Consortium/Community Health Center Network','Courtney Raymond Consultants','Facing History and Ourselves, Inc thrivecausemetics.com','BI Worldwide','Advanced ICU Care','AnaVation, LLC','Ingram Content Group','Twitch','Renown Health','DXC Central Health','Western National Insurance Group','AmeriCorps','Gaylord Specialty Healthcare','Housing Works Comprehensive Healthcare','Stormont-Vail Healthcare','Bassett Healthcare','Fiserv','Holleran Consulting','Nestle USA','Payfone University of Pennsylvania','Concerted Care Group','KingsIsle Entertainment','Community Reach Center','The Boston Globe Volvo Group','UNITE HERE HEALTH','2DA Analytics','ForeFlight','Care.org','Tory Burch','GEHA','Buxton','Hylink Group','alliantgroup Culinary Health Center','The Vancouver Clinic','University of Virginia Health System','Cancer Treatment Centers of America Inframark','Jamul Casino','ipsy','Medifast, Inc','American Merchandising Specialists','Success Academy Charter Schools Compass Group USA','Webster Bank','John Clements','ANB Bank','L'OREAL USA','Association of Universities for Research in Astronomy Sullivan, Cotter and Associates, Inc.','Primus Builders Inc.','Medical Transportation Management (MTM) National Financial Partners','Fragomen','Fresenius Medical Care','Evolent Health','Renewable Energy Systems Ltd. Hawaii Medical Service Association','Child Care Resource Center ‚Äì Chatsworth, CA','Aultman Health Foundation D&K Engineering','Axos Bank','Churchill Mortgage Corp','Dexperts','NSD','Primerica','EmblemHealth Health Care District of Palm Beach County','Baystate Health','AccessCNY, INC','Emerson Hospital The International Fund for Animal Welfare','Acquisition Life Cycle Management','Aultman Hospital','AUTHENTIX INC Flair IT Solutions','Boston Children's Hospital','Wood','Targetbase','Edward-Elmhurst Health','Allegheny Health Network','EMA, Inc. MJHS','Aclara','Boyne Resorts','US Software & Consulting','Encompass Community Services','Everything But The House (EBTH) Arc Aspicio','Alnylam','Team Red Dog','Orange County Transportation Authority','Rauxa','Direct Resources Group','Johnson Matthey Vivo','Healdsburg District Hospital','Dime Community Bank','CIBC','Dassault Falcon Jet','Emmis Communications','Logic20/20 CORKCICLE.','Virginia Jobs','Logitech','BBYO','Healthline Media','CSRA','University of Texas at Arlington','Crestron Electronics Crestron','University of Colorado','C5T Corporation','TISTA Science and Technology Corporation','Clearwater Paper','Option Care University of Texas at San Antonio','Knorr-Bremse North America','Bird Rides Inc.','Maryville University','Listrak State of New Mexico','Management Science Associates Inc.','Managed Care Advisors','Flight Centre','RWJBarnabas Health Beaumont Health','Envisagegroup','Ensign Services','Sweetser','The Parking Spot','Zodiac Aerospace','AIDS Foundation of Chicago Baptist Health South Florida','Fabick Cat','Community Behavioral Health','Academy for Urban School Leadership Arrowpoint Corporation','Chicago Public Schools','AUSL Chicago','Gila River Health Care','SECU','Charles Schwab','MultiPlan Inc. Chapman Cubine Adams & Hussey','Management Concepts','NaviHealth','thoughtmatrix','ClassLink Inc','AHMC Healthcare Inc','RE/MAX, LLC Capital Impact Partners','Clayton Homes','United Fire Group','MIT Lincoln Laboratory','BookBub','AssuredPartners','PowerSchool The Situs Companies','Mary Washington Healthcare','Brightree','ITAGroup','Virginia Mason Medical Center Appalachian Regional Healthcare','MaineHealth','Horizon Health Services','Nationwide Children's Hospital','Hitech Assets, LLC. NetVision Resources (NVR)','Kroger','Penobscot Community Health Care','Unilever','NRG Energy','State of Arizona Laureate International Universities','QuEST Global Engineering','ITT Corporation','Citizens Energy Group LiniumTalentAcquistion','Nolij Consulting','ACN','Blue Cross Blue Shield of Arizona Advantage','Wildlife Conservation Society Carter Bank and Trust','Astor Services For Children & Families','2U','Systems Staffing Group','Zumba Fitness, LLC','BOK Financial CHEP','Harvard Pilgrim Health Care','Credit Acceptance','Sun Life Financial','8-Koi','The New School','Curriculum Associates','Dyson Commonwealth Of Virginia','Universal Health Services','Premera Blue Cross','Baltimore Medical System, Inc. University of California Berkeley','Fossil Group','Fund for Public Health in New York City','Indiana University','Siena College Precision Value & Health','Piedmont Healthcare','Children's Hospital & Research Center Oakland','GrayMatter','Parametric Gray Matter Systems','Partners HealthCare','Anchorage Consultants LLC','Advantage Solutions','Cydecor, Inc. Toole Design Group LLC','FARO Technologies','Blue Shield of California','Kongregate','TracyLocke','Florida Hospital Adventist Health System','Geotab','Shutterfly','Rosecrance Health Network','Morneau Shepell','Farm Bureau Insurance of Michigan Pace University','Einstein Healthcare Network','SUBWAY','EnerNOC','Cricut','The Money Source','Santander Consumer USA Nordic Naturals, Inc.','ProMedica','Akima, LLC','Freddie Mac','Northeastern University','Beaver Dam Community Hospitals, Inc. Discover Financial Services','Cloudburst Consulting Group Inc','Crozer Keystone Health System','Capgemini Government Solutions CompuGain','Envision Healthcare','Molina Healthcare','Epic Care','Calero Software','CCC Information Services Inc.','CBRE Freedom Financial Network','Self Regional Healthcare','BlueLinx Corporation','KGPCo','LTD Commodities UT Southwestern Medical Center','Frogdata','The Logistics Company','Imerys','T. Parker Host','Gallagher','maconit','VCU Health System Mount Sinai Health System','Benchmark Education Company','Oxford Global Resources','Astronics','Accountants One, Inc','Ameritas Multnomah County, OR','Mercy Health','Nextiva','Parker Hannifin Corporation','Northwest Permanente, P.C. Matheny School and Hospital','Na Ali'i LLC','The National Association of Manufacturers','Vista Technology Services Liberty University','Anheuser-Busch','AbleVets LLC','Swiss Re','Cymer','Boston Medical Center','Baylor Scott & White Health','Bitcoin Under Armour','Creative Arts Agency (CAA)','Voloridge Investment Management','Dot818','Quinstreet','Cisco Meraki Guardian Industries','Inland Empire Health Plan','Plastiq','Cricket Health','InnovaSystems International','OneAmerica','Techshed Core10','Leadsmarket.com LLC','Fiat Chrysler Automobiles','Aramark','Cognosante, LLC','OppLOans','Fidelis Care','Fullscreen Media Pond5','Ring Inc.','Columbia Sportswear','Manage','Santa Clara Family Health Plan','LawTrades','New York Blood Center','Lucky Day Buchanan & Edwards','Vox Financial Partners','Doximity','Candid Partners','Dentsu Aegis Networks','MKTG','ZeniMax Media Inc. University of Washington','Fullscreen','Polaris Industries','Cogo Labs','The New York Times','RiskSpan','Poll Everywhere NCI Information Systems, Inc.','FPM Technologies','enVista','Cable & Wireless Communications Inc','Dewpoint Northwest Farm Credit Services','IMTAS','Textio','Apptentive','Ascend Performance Materials','Apkudo Prospect Medical Holdings, Inc.','Prospect Medical Systems','Greater Delaware Valley','Silicon Valley Bank','IDT Corporation Celmatix','University of Michigan','FourKites','CONTINUUM','Hilcorp Energy Company','Innove LLC','ResMed','Ellis Medicine Relevant Healthcare Technologies','Hasbro Inc.','UCLA Health','Corcentric','Mentor Graphics','Ingram Micro','Curve IT Consulting GCC Technologies, LLC','AppFolio','Langan Engineering and Environmental Services, Inc.','Legg Mason','Sentara Healthcare Reputation.com','Institute for Health Metrics and Evaluation - Faculty','CliniWorks','Great Wolf Lodge','Stream','IDEO iSpot.tv, Inc.','CHOC Children's Hospital','SocialWithin','Community Medical Centers Inc','Harry's','Verscend Technologies Zelis Healthcare','LevelUp','Delta Dental of Minnesota','Community Medical Centers','Kohler Co.','Galaxy Systems','Asembia LLC Common Securitization Solutions','Tate & Lyle','Ascensus','Getty Images','ddms','Emory Healthcare','Louis Dreyfus Central Maine Medical Center','InfoArmor Inc','Prime Healthcare','Castleton Commodities International University of California UCOP','Collective Health','Fanatics Inc.','T1D Exchange','sweetgreen','Brooks Sports, Inc','Virtru','JFrog Bank of the West','Hiscox Ltd','State Street','Meijer','Kabam','Superior Vision','Archer Daniels Midland Company South Dakota State University Foundation','Starbucks','Arthrex','Spireon','PitchBook Data, Inc.','Commonwealth Care Alliance Donnelley Financial Solutions','CORTEK','VSCO','William E. Wecker Associates, Inc.','Rent-A-Center','New York University Russian School of Mathematics','Din√© Development Corporation','A. H. Belo Corporation','Vera Whole Health','Assurant Strategic Resources, Inc.','Beachbody','Clarkston Consulting','Riot Games','College Raptor','REI','Boxy Charm','Acumen Solutions Pinnacle Engines, Inc.','Glu Mobile','CenCal Health','Ely-Bloomenson Community Hospital','National Instruments','BorderX Lab Inc Central Texas Food Bank','MileOne Automotive','Clemson University','GenapSys, Inc.','Paradyme Management','Coffee & Bagel Brands Aledade, Inc.','Blue Chip Talent','Blue Apron','HelloWorld','Straumann','Libra Services','TSheets','LUXOTTICA','Mindshare','GoodRx','JT4 Berkadia Commercial Mortgage','IHS','IHS Markit','State of Washington','Cambria','Level Ex, Inc.','MUFG','Fremont Bank DSD Partners Inc','Genworth','C&A Industries, Inc','Crown Castle','Union Bank & Trust','CSSI, Inc.','RK','Advance Financial Halfaker and Associates','EBSCO Industries Inc','Arbella Insurance Group','Catholic Health Initiatives','ASML','T. Rowe Price','SAIC Washington State Hospital Association (WSHA)','Vertical Careers, Inc.','L.A. Care Health Plan','George Mason University Willis Towers Watson','Providence Health & Services','AmeriHealth Caritas','New York State Office of the Attorney General RentPath, LLC','Los Alamos Technical Associates','PANGEATWO','Seasoned','Christiana Care Health Systems','Niagara Bottling Allegro MicroSystems, LLC','Virtus Partners','Wargaming.net','Johns Manville','Life Chiropractic College West Federal Reserve Bank of New York','Meridian Health Plan','DCS Corp','Digital Management, LLC','OneSource Virtual Genuine Parts Company','Wyndham Hotels & Resorts','Vitals','Green Dot Corporation','UJA Federation of New York','Inovalon Alliance Data - Retail','FirstCare Health Plans','SunPower Corporation','Solomon Associates','ANTHEM MARKETING Children's Hospital of Wisconsin','Capital One','Mountain America Credit Union','DJO','Radiance Technologies Inc.','HBM Holdings Smith & Nephew','VirtueGroup','The American College of Radiology','Purple Communications, Inc.','CapTech Consulting','SRC, Inc. First San Francisco Partners','AnalogFolk','NinthDecimal','KeepSafe','Periscope technologies inc','Health Team Advantage Bowery Farming','Alteryx, Inc.','Hudson's Bay','Traxion Group, Inc','Foundation Medicine, Inc.','Coty Inc.','QIC Limited Nextphase Systems','Hearsay Social','EPMA','Certara','Women's World Banking','Microfinance Gateway','Impact Makers, Inc.','Balyasny TACG, LLC','AppNexus','Health Dialog','Burlington Stores','ARC Solutions USA','LeafLink','Productive Edge','Credit Suisse General Dentistry 4 Kids','Nuvve','Harvard Business Publishing','Grail Insights','Heartland Business Systems','RocksBox Crohn's & Colitis Foundation of America','goPuff','Layer3 TV','Real Capital Analytics (RCA)','Scholastic','IntroPro','MWW Group ExcelaCom','One Call','Seattle Children‚Äôs','Estee Lauder','[24]7','PeerStreet','WeWork','Virbac Animal Health Mississippi Lime Company','County of San Mateo','Softrams','BLUECUBE Information Technology','CooperVision','IPG Mediabrands','ALC Accolite Software India','Accolite, Inc.','General Motors','BNY Mellon','PJM Interconnection, LLC','WP Engine','Priceline.com Oklahoma Cancer Specialists and Research Institute','Blytheco, LLC','Johns Hopkins Health System','momofuku','RB','Alarm.com ServiceLink','Becton Dickinson','Abacus Data Systems','City National Bank','LendingClub','MGAC','Kennametal','Allergan','MVM','Magento Mission Staffing','HelloFresh','CTG','Health First','MM.Lafleur','Chicago Trading Company (CTC)','National Funding','Looker','Clearpoint uJoynus','CAPCO','Fender','ClearView Healthcare Partners','SWIFT','Bill.com','Motion Recruitment Partners','Publicis.Sapient','PureCars Arch Capital Group Ltd.','Engage Partners','Western Union','Community Health Network','Broadridge','Robert Walters','Merkle Inc. L Brands','Sephora','King.com','HAVI Logistics','Globoforce Limited','Wycliffe Bible Translators','Vistage Worldwide','Wipro LTD Franciscan Health','Coffee Meets Bagel','Zenith','Finra','Ramsey Solutions','Medidata Solutions','J.D. Power','JustAnswer','Carbon Black Performics','JBCconnect','Goodwin Recruiting','DoorDash','MSCI Inc.','Jetblack','EyeView','New York Yankees','EducationDynamics Boston Bruins','State of North Carolina','The University of Kansas Health System','American Specialty Health','Hydromax USA Hill-Rom','Solomon Page','cableyou','Hy-Vee, Inc.','ATN International','Measured Progress, Inc.','Availity, LLC. Arrow Electronics, Inc.','Colorado Community Managed Care Network','Koddi','ELLKAY LLC','A Could Guru','Janus Research Group Quest Analytics','Bealls Inc','Technamo LLC','Eastman Chemical Company','Evonik','Virginia Tech','Renew Financial','Aroghia Group PRA Group','Advanced Technology Group Inc','UpClear','Acustream','Beacon Health System','Grundfos','Grundfos Pumps Corporation','Snag WEX Inc.','East Daley Capital Advisors','Denodo Technologies','AXA General Insurance','Bangor Savings Bank','AQR','AXA','Sierra Lobo Housing Authority','Munich Re','Las Vegas Sands Corp.','BGC Partners','Denodo','Jackson Dawson','Henry Schein','ECRI Institute American Modern Insurance Group, Inc.','Octo Consulting Group','Serco , Inc.','ACT, Inc.','TeleTracking Technologies Farmers Insurance Group','Auth0','Liv Communities','Blue Cross Blue Shield of Arizona','Prosearch Strategies Casey's General Stores','Prodege, LLC','Hook & Loop Creative','ktMINE','Audley Travel','AKRAYA INC.','Viant','SharpSpring','ACCION Pairwise','Accion in Arizona, Colorado, Nevada, New Mexico and Texas','Clear Capital','Catalyst Repository Systems Billings Clinic','Doosan Bobcat','Intrexon Corporation.','Tredence','GlobalLogic, Inc.','Intalere Inc','Rackspace','Nevro Corporation Tompkins International','Asics Digital','The Walsh Group','Object Computing Inc.','Lirio','Paycom','Atlantic Health System VVC Holding Corp','Curbside','Susquehanna International Group','Plante & Moran','State Auto Insurance Companies','Overstock.com Farmers Fridge','Southern Poverty Law Center','Privia Health','Pizza Hut','United States Associati','InsideSales.com, Inc. REsurety','Jebbit','Civitas Learning','Auto Club Group','Hamilton Lane','Navigator Management Partners','Choice Hotels','SailPoint AmeriSourceBergen','Theoris Services','Carbonite','Jellyfish UK','Jellyfish','Tabula Rasa Healthcare, Inc','Morningstar','Armor WebbMason Analytics','BCD Travel','Zilker Partners','Simon-Kucher & Partners','Syngenta','Aegon','Cross River Bank Dev Technology Group, Inc.','W.W. Grainger','Calix','Healthfirst','Mondelƒìz International','Vivint Smart Home','IDEXX','New Knowledge Location3','Windstream Communications','Windstream','Trident Technologies LLC','United Network for Organ Sharing','Penske Macquarie Group Limited','Idexx Laboratories','Schoolzilla','CoreCompete','Netsmart Technologies','Rally Health','Lifetouch Hospital for Special Surgery','TMP Worldwide','The Denzel Group','MicroStrategy','Vets First Choice','PATHORAS CORPORATION','Nice Sumeru Solutions','Cimpress','Brivo Systems','Caesars Entertainment','IMO - Intelligent Medical Objects, Inc. LifeWatch Services, Inc.','BioTelemetry, Inc.','Fannie Mae','The Standard','RhythmOne','Whole Foods Market','Kraken Cleveland Cavaliers','Zeus','LeanTaas','Hulu','BitTorrent Inc.','7-Eleven','Tavant Technologies','Orzota','Reflektive Anadarko Petroleum Corporation','Zeus Living','Blizzard Entertainment','HelloSign','Pandora Media, Inc.','The Black Tux','8X8 INC. Exabeam','N3TWORK','VividCortex','Samsung','US Information Technologies','Gallup','Linden Lab','Brad's Deals Perfect World Entertainment','AirPR','GoodData Corporation','UiPath','Castle Global','Duke Energy','Principal Financial Group','Trianz Warby Parker','Syneos Health Commercial Solutions','Evaya Data Systems','Sun Basket','Drillinginfo','TGS Management Company','Novus Caserta','hiretual.com','ChowNow','Thales','Peloton Technology','St. Joseph Health','SoftVision - North America & UK Keysight Technologies','Mazda North American Operations','Infront Sports & Media Group','Amdocs','14 West Amplify Education, Inc.','Animoto','Change.org','Agent IQ','Zipcar','Citadel Securities','PlacePass','J. Crew Group, Inc. Augmedix Inc.','Myers Media Group, LLC.','SmartThings','Object Partners','Aaptiv','Hinge Health','Swift Navigation Public Consulting Group','Plus3 IT Systems','Pythian','MLB.com','ScienceSoft USA Corporation','NeuStar, Inc.','SSi','ThoughtWorks Criterion Systems','USANA Health Services','USANA Health Sciences','Tendril','Takeda Pharmaceuticals','Softvision University of Maryland, Baltimore','Ten-X','National Home Rentals','Federal Reserve Bank of St. Louis','Sevatec, Inc. athenahealth','Integrity Applications Incorporated','Lucid Motors','TE Connectivity','NISSAN','GoPro','HatchWorks Technologies Merlin Labs','Tableau','Giant Oak','Yodlee','Probity Inc.','Eaton','Zotec Partners','PlayQ','KEYPR','GumGum','drawbridge','Scoop Technologies HotSchedules','iRobot Corporation','National Football League','VideoAmp','Alloy','EquiLend','Viacom','TPC Energy Fund','Shippo INSIGHT ENGINES','Copper','lululemon athletica','Faraday Future, Inc.','William Blair & Company','Applied Thought','Custoria','ZocDoc The Zebra','Symphony Talent','Mercari','Credit Karma','Levi Strauss & Co.','EasyPost','Carrot Inc.','zesty.ai','WeedMaps The Clorox Company','Omada Health','Enlighted','Adaptive Management','Cyberonics','Smule','Boeing Intelligence & Analytics Celtra Inc.','AG Grace Inc','Obsidian Security','thredUP Inc','Guardant Health','Audi Canada','FuelX','Chegg','Hitachi','StreetLight Data Edwards Lifesciences','DATA WORKS','Mojio','Axis Group','A+E Networks','Navy Federal Credit Union','Concur','M&T Bank','East West Bank Vistaprint','Veteran's Enterprise Technology Solutions, Inc.','Dialpad, Inc.','System1','realtor.com','AppDynamics','Conductor Stessa','BigCommerce','Avvo','The Meet Group','Glooko','Coda Search','Intelliswift Software Inc']
  ''' 






"            \n                    'University of California San Francisco','Wunderman','MGM Resorts International','Interactive Brokers','Grubhub Amica Insurance Company','UPMC','Hearts and Science','Operation Smile','Sedgwick Claims Management Services','MetLife'\n                       ,'Code Pilot','Talking Rain Beverage Company','Bright Cellars','MetroStar Systems','Omatic Software Schnucks','Abbott Laboratories','Lawrence Berkeley National Laboratory','Shift Technology','Consumers Energy Penn State University','VillageCare','Intercontinental Exchange','Palo Alto Networks','The Rawlings Group','Devoted Health Caris Life Sciences','ADT Corporation','Thomas Jefferson University and Hospitals','MassMutual Financial Group FourthWall Media','BB&T','AbbVie','National Renewable Energy Laboratory','Sandhills Publishing',\n                       'Energy Sense Finance New York Media LLC','ESAC Inc','Aunalytics','U.S. Bank','Vistra Energy','Precision for Medicine','Teradata','Greene, Tweed &

In [79]:
#jd_sentences = open("./jd_sentences.txt").read()
job_description = open("./cleaned_jd.txt").read()
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1123)>


In [80]:
jd_sentences = sent_tokenize(job_description)
jd_sentences[0:255]

['POSITION SUMMARY, \n\nThe Business Analyst role is the primary architect of reporting and dashboard solutions for internal and external clients.',
 'Utilizing ESI corporate standard development tools this position is responsible for the design, development, implementation, analysis, interpretation and communication of business information based on the needs of individual clients.',
 'The ability to balance overall aesthetics with robust and intuitive functionality is a critical requirement for success in this position., \n\nESSENTIAL FUNCTIONS, \n\nSuccessfully design and implement external client data reporting and dashboard solutions with a strong focus on product aesthetics and functionality.',
 'Aid in the design, development, and implementation of new product ideas for external and internal clients.',
 'Maintain Live and Data Warehouse Business Objects Universes; add new fields, modify table joins, implement data structures that streamline report extraction and data analysis.',


In [85]:
def generate_corpus(sentence, words):
    unique_description = []
    res = []
    for entry in words:
       # entry = entry.strip()
        for substring in sentence:
           # print(substring)
            if entry in substring:
           # if (len(k) == len(words) ):
                print ('entry --- ' + entry)
                print ('substring --- ' + substring)

                res.append(substring)
            
    unique_description = set(res)
    unique_description = list(unique_description)
    return unique_description
      
# Driver code
sentence = ['python coder SQL in PayPal design', 'geeksforgeeks', 'SQL']
words = ['PayPal','role']
jd_rol_res_corpus = generate_corpus(jd_sentences,company_keyword_seed)
print(jd_rol_res_corpus)

entry --- Express Scripts
substring --- , 

Bachelor‚Äôs degree in related field or 8 to 11 years of experience., 

ABOUT THE DEPARTMENT, 

ABOUT EXPRESS SCRIPTS, 

Advance your career with the company that makes it easier for people to choose better health., 

Express Scripts is a leading healthcare company serving tens of millions of consumers.
entry --- Express Scripts
substring --- We offer a highly competitive base salary and a comprehensive benefits program, including medical, prescription drug, dental, vision, 401(k) with company match, life insurance, paid time off, tuition assistance and an employee stock purchase plan., 

Express Scripts is an equal opportunity employer/disability/veteran]"
"[What do we need?, 

You to have an amazing personality and communication style.
entry --- Express Scripts
substring --- , ABOUT EXPRESS SCRIPTS, 

Advance your career with the company that makes it easier for people to choose better health., 

Express Scripts is a leading healthcare comp

entry --- ExxonMobil
substring --- Experience in Python, Scala, Java, C, C++ or R is required
Knowledge of statistics, linear algebra, multiple variable calculus, Fourier analysis or machine learning
Experience using one or more of the following software packages: scikit-learn, numpy, pandas, jupyter, matplotlib, scipy, nltk, spacy, keras, tensorflow
Experience solving problems using one or more of the following techniques: Regression, Support Vector Machines, Decision trees, random forest, Boosting, PCA, KMeans
Experience in using SQL/No SQL databases is an advantage
Experience working in Linux and in a High Performance Computing environment is an advantage
, Alternate Location: United States : Baytown, Texas || United States : Clinton, New Jersey || United States : Hugoton, Kansas, ExxonMobil is an Equal Opportunity Employer.
entry --- ExxonMobil
substring --- spatial pattern extraction and analysis in 2D and 3D)., Responsibilities:, Apply statistical analysis, pattern recognition, a

entry --- Google
substring --- Experience using Google Analytics and Google Tag Manager is a plus.
entry --- Google
substring --- Expert user of Microsoft and Google office suites.
entry --- Google
substring --- , JOB SPECIFIC KNOWLEDGE, SKILLS, AND EXPERIENCE, Bachelor‚Äôs degree in a highly analytical field such as marketing analytics, mathematics, economics, finance, information systems, or statistics
3+ years of experience providing data insights to a marketing team
Experience with creating predictive statistical models
Experience writing SQL, MySQL, and Python scripts
Knowledge of SAS, R, or another language with statistical capabilities
Experience utilizing data from a variety of internal and external sources including, but not limited to: Google AdWords and Analytics, Microsoft Dynamics CRM, Marketing automation platforms, AWS, third-party databases, etc.
entry --- Google
substring --- The role serves as a key Marketing Analytics team member in measuring performance of marketing

entry --- Google
substring --- Experience in Google Cloud Services strongly preferred.
entry --- Google
substring --- Experience in Google Cloud Services strongly preferred.
entry --- Google
substring --- Experience with Agile methodologies (Scrum)

Experience working with large data processing platforms like Apache Beam, Apache Spark, or Apache Hadoop MapReduce (Google Cloud Dataflow a plus).
entry --- Google
substring --- Experience with Google Analytics is a plus

Strong technical aptitude and a willingness to learn and adapt to new and evolving technologies

Excellent communication skills

Team Skills: facilitation, presentation & group dynamics, 

Much has changed since our start in 1912, but the important things remain the same.
entry --- Google
substring --- "[
3 years of experience with Big Data, systems, including Hadoop, Hive and Pig
Experience with ETL tools including but not limited to NiFi and StreamSets
Experience with Java
Experience with using Cloud services, including 

Effective July 1, 2014, under Indiana House Enrolled Act (HEA) 1242, it is against public policy of the State of Indiana and a discriminatory practice for an employer to discriminate against a prospective employee on the basis of status as a veteran by refusing to employ an applicant on the basis that they are a veteran of the armed forces of the United States, a member of the Indiana National Guard or a member of a reserve component.
entry --- Twitter
substring --- We engage and develop people to their greatest potential., 

Work Collaboratively: We work together to achieve results by actively listening, seeking, understanding and creating solutions as a unified team driving toward one company, one culture, one brand., 

Achieve Results: We focus on winning by exceeding expectations and getting better ‚Äì everyone, every day., 

For more information, visit www.massmutual.com or find us on Facebook, Twitter, LinkedIn, YouTube, Google+, Instagram and Pinterest., 

MassMutual is an Equal

substring --- Experience with source control solutions (ex git, GitHub, Jenkins, Artifactory) required
At least 1-2 years of experience with big data and the Hadoop ecosystem (HDFS, SPARK, SQOOP, Hive, Impala, Parquet) required
Experience with Agile development methodologies and tools to iterate quickly on product changes, developing user stories and working through backlog (Continuous Integration and JIRA a plus)
Experience in any scheduling tools such as IBM Tivoli Workload Scheduler, Control M, AutoSys, etc required
Must be a self-starter and have excellent oral and communication skills
, The candidate(s) offered this position will be required to submit to a background investigation, which includes a drug screen., Good Work.
entry --- IBM
substring --- Experience with source control solutions (ex git, GitHub, Jenkins, Artifactory) required
At least 1-2 years of experience with big data and the Hadoop ecosystem (HDFS, SPARK, SQOOP, Hive, Impala, Parquet) required
Experience with Agil

substring --- Our vibrant and diverse international community of nearly 250 publishing brands and imprints include Ballantine Bantam Dell, Berkley, Clarkson Potter, Crown, DK, Doubleday, Dutton, Grosset & Dunlap, Little Golden Books, Knopf, Modern Library, Pantheon, Penguin Books, Penguin Press, Penguin Random House Audio, Penguin Young Readers, Portfolio, Puffin, Putnam, Random House, Random House Children‚Äôs Books, Riverhead, Ten Speed Press, Viking, and Vintage, among others.
entry --- Dell
substring --- If you have what it takes to bring innovative new products and services to life in collaboration with world-class experts, this is your opportunity to develop with Dell.
entry --- Dell
substring --- Dell is committed to the principle of equal employment opportunity for all employees and to providing employees with a work environment free of discrimination and harassment.
entry --- Dell
substring --- All employment decisions at Dell are based on business needs, job requirements and 

In [86]:
text_file = open('./company_corpus_V1.txt','w')

word_list= []

for i in jd_rol_res_corpus:
    line = i 
    word_list.append(line) 
text_file.writelines(word_list)
 
text_file.close() 